## Benchmark sparse matrices 

In [1]:
import scipy.sparse as sp
import numpy as np

m,n = 4000, 3000

A_lil = sp.random(m, n, density = .2, format='lil')
A_csc = sp.csc_matrix(A_lil)
A_csr = sp.csr_matrix(A_lil)
A_bsr = sp.bsr_matrix(A_lil)
A_coo = sp.coo_matrix(A_lil)

b = np.random.randn(n)
c = np.random.randn(m)

In [2]:
#%timeit A_coo@b

In [3]:
from numba import jit

@jit(nopython=True)
def csr_matvec(row_pointers, col_indeces, mat_elements, vector):

    m = len(row_pointers) - 1
    result = np.zeros(m)
    
    for i in range(m):
        js = col_indeces[row_pointers[i]:row_pointers[i+1]]
        elements = mat_elements[row_pointers[i]:row_pointers[i+1]]
        for cur, j in enumerate(js):
            result[i] += vector[j] * elements[cur]
    
    return result  

@jit(nopython=True)
def csc_matvec(m, col_pointers, row_indeces, mat_elements, vector):
    result = np.zeros(m)        
    n = len(col_pointers) - 1
    assert len(vector) == n
    for j in range(n):
        i_s = row_indeces[col_pointers[j]:col_pointers[j+1]]
        elements = mat_elements[col_pointers[j]:col_pointers[j+1]]
        for cur, i in enumerate(i_s):
            result[i] += elements[cur] * vector[j]
    
    return result 

#csc_matvec(A_csc.shape[0], A_csc.indptr, A_csc.indices, A_csc.data, b)

In [4]:
%timeit A_csc@b

2.73 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%timeit csc_matvec(A_csc.shape[0], A_csc.indptr, A_csc.indices, A_csc.data, b) 

3.18 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
csc_matvec(A_csc.shape[0], A_csc.indptr, A_csc.indices, A_csc.data, b) 

array([-30.6177412 ,   8.91547711, -26.82965844, ..., -26.87710068,
       -34.40569048,  25.37843367])

In [22]:
np.all((A_csc.T @ c) == csr_matvec(A_csc.T.indptr, A_csc.T.indices,  A_csc.T.data, c) )

True

In [33]:
%timeit csc_matvec(A_csc.shape[0], A_csc.indptr, A_csc.indices, A_csc.data, b)

14 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%timeit A_csr@b

3.3 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
#len(A_csr.indptr)

In [28]:
%timeit csr_matvec(A_csr.indptr, A_csr.indices, A_csr.data, b)

3.65 ms ± 245 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
#%timeit A_lil@b

3.14 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
#%timeit A_bsr@b

37.6 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
#np.var(csr_matvec(A_csr.indptr, A_csr.indices, A_csr.data, b) - A_csr @ b)

In [30]:
m,n = 6,4
A = sp.random(m, n, density = .2, format='csc')
b = np.random.randn(n)

print(A.todense())

csc_matvec(m, A.indptr, A.indices, A.data, b) - A@b

[[0.30871792 0.         0.         0.76990713]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.12435293 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.85768312 0.        ]]
[0]
[0.30871792]
0
[3]
[0.12435293]
3
[5]
[0.85768312]
5
[0]
[0.76990713]
0


array([0., 0., 0., 0., 0., 0.])

In [97]:
m,n = 6,4
A = sp.random(m, n, density = .2, format='csc')
b = np.random.randn(n)
print(A.indptr)
print(A.indices)
print(A.data)
A.todense() #b

[0 1 3 4 4]
[1 3 4 2]
[0.79031659 0.78557056 0.31286768 0.42342571]


matrix([[0.        , 0.        , 0.        , 0.        ],
        [0.79031659, 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.42342571, 0.        ],
        [0.        , 0.78557056, 0.        , 0.        ],
        [0.        , 0.31286768, 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ]])

In [54]:
m,n = 5, 10
A = sp.random(m, n, density = .2, format='csc')
b = np.random.randn(n)
csr_matvec(A.shape[1], A.indptr, A.indices, A.data, b) -A@b

js []
ele []
js [7 8]
ele [0.10888623 0.41501452]
js [0 2]
ele [0.3266154  0.78781857]
js [5]
ele [0.47700821]
js [2 3 5 7 8]
ele [0.54040142 0.25253436 0.40707859 0.20055761 0.08193714]


array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.11022302e-16])

In [20]:
row_pointers, col_indeces, mat_elements = A.indptr, A.indices, A.data
m,n = A.shape

csr_matvec(A.shape[1], A.indptr, A.indices, A.data, b)

js [4]
ele [0.9899777]
js [4]
ele [0.33182386]
js []
ele []
js [1 5]
ele [0.77959195 0.26700234]
js [1 5]
ele [0.8908724  0.45704714]


array([-0.10062724, -0.03372856,  0.        ,  0.70919177,  0.92476466])

In [21]:
A@b

array([-0.10062724, -0.03372856,  0.        ,  0.70919177,  0.92476466])

In [9]:
print(A.indptr)
print(A.indices)
print(A.data)
print(A.todense())

[0 1 2 4 4 6]
[5 5 2 3 1 2]
[0.29980696 0.0332915  0.45539075 0.11370969 0.04820431 0.75973036]
[[0.         0.         0.         0.         0.         0.29980696]
 [0.         0.         0.         0.         0.         0.0332915 ]
 [0.         0.         0.45539075 0.11370969 0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.04820431 0.75973036 0.         0.         0.        ]]


In [169]:
#np.dtype.char

In [11]:
# from scipy.sparse import _sparsetools
# from scipy.sparse.sputils import upcast_char

# result = np.zeros(m)#, dtype=upcast_char(np.dtype.char, np.dtype.char))
# m,n = A.shape
# _sparsetools.csc_matvec(m,n, A.indptr, A.indices, A.data, b, result)
# result

In [10]:
#print(b)
print(A@b)

[-0.3885075  -0.04314108 -0.45497864  0.         -0.98281156]


In [148]:
M,N = A.shape
_sparsetools.csc_matvec(M, N, self.indptr, self.indices, self.data, other, result)

In [140]:
#A_csc.todense()

In [59]:
import numba as nb

@nb.jit(nb.double[:](nb.double[:,:]))
def flatten(mat):
    return mat.flatten()

@nb.jit(nb.double[:,:](nb.double[:]))
def reconstruct_sqmat(vec):
    n = int(np.sqrt(len(vec)))
    return vec.reshape(n,n)

np.all(reconstruct_sqmat(flatten(A)) == A)


True

# Test

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
import cone_prog_refine as cpr 
import time

import cvxpy as cvx
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
n = 20

A = cvx.Variable((n,n))
B = cvx.Parameter((n,n))
B.value = np.random.randn(n,n)
obj = cvx.norm(A - B)

constr = [
    A == A.T,
    A >=0,
    A >> 0,
    cvx.log_det(A) >= 2.,
]

problem = cvx.Problem(cvx.Minimize(obj), constr)

In [3]:
#problem.solve(verbose=True)
#mosek_opt = problem.value

In [4]:
# np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)), \
# np.log(np.linalg.det(A.value)) - 2

In [5]:
from cvxpy.reductions.solvers.solving_chain import construct_solving_chain

def cvxpy_scs_to_csr(data, scs_solution = None):

    A, b, c, dims = data['A'], data['b'], data['c'], data['dims']
    
    #z_scs = cpr.xsy2z(scs_solution['x'], scs_solution['s'], scs_solution['y'])
    if scs_solution is None:
        z_scs = np.ones(len(b) + len(c) + 1)
    else:
        z_scs = np.concatenate([scs_solution['x'], scs_solution['y']- scs_solution['s'], [1.]])

    
    dims_dict = {}
    dims_dict['l'] = int(dims.nonpos)
    dims_dict['z'] = int(dims.zero)
    dims_dict['ep'] = int(dims.exp)
    dims_dict['q'] = list([int(el) for el in dims.soc])
    dims_dict['s'] = list([int(el) for el in dims.psd])
    
    
    return A, b, c, z_scs, dims_dict

def solve_and_refine(cvxpy_problem, presolve=True, iters = 2, lsqr_iters = 30, verbose = True, warm_start=True):
    solving_chain = construct_solving_chain(cvxpy_problem, solver = 'SCS')
    data, inverse_data = solving_chain.apply(cvxpy_problem)
    
    if presolve:
        scs_solution = solving_chain.solve_via_data(cvxpy_problem, 
                                                    data=data, 
                                                    warm_start=warm_start, 
                                                    verbose=verbose,
                                                    solver_opts = {})#'eps':1E-100})

        #assert problem._solver_cache[list(problem._solver_cache.keys())[0]] is scs_solution
        A, b, c, z_scs, dims = cvxpy_scs_to_csr(data, scs_solution)
    else:
        A, b, c, z_scs, dims = cvxpy_scs_to_csr(data)
        scs_solution = {}
    #TODO change this
    if warm_start and 'CPSR' in problem._solver_cache:
        z_scs = problem._solver_cache['CPSR']['z']
        
    ## HERE GOES REFINE
    refined_z = cpr.refine(A,b,c, dims, z_scs, iters = iters, lsqr_iters=lsqr_iters, verbose=verbose)
    #refined_z = cpr.ADMM_refine(A,b,c, dims, z_scs, iters = iters, lsqr_iters=lsqr_iters, verbose=verbose, U_UPDATE=1)
    problem._solver_cache['CPSR'] = {'z':refined_z}
    new_residual, u, v = cpr.residual_and_uv(refined_z, (A.indptr, A.indices, A.data), b, c, cpr.make_prod_cone_cache(dims))
    scs_solution['x'], scs_solution['s'], scs_solution['y'], tau, kappa = cpr.uv2xsytaukappa(u, v, A.shape[1])
    scs_solution["info"] = {'status':'Solved', 'solveTime':0.0, 'setupTime':0.0, 'iter':iters, 'pobj':np.nan}
    problem.unpack_results(scs_solution, solving_chain, inverse_data)

    return problem.value

In [34]:
B.value += np.random.randn(n,n) * 1E-5

In [35]:
problem.solve(verbose=True, eps=1E-10)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 7190, CG tol ~ 1/iter^(2.00)
eps = 1.00e-10, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 2481, constraints m = 5531
Cones:	primal zero / dual free vars: 3010
	linear vars: 401
	sd vars: 2060, sd blks: 4
	exp vars: 60, dual exp vars: 0
Setup time: 1.75e-03s
SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.06e-06  1.06e-06  2.54e-07  7.07e+00  7.07e+00  0.00e+00  4.07e-03 
   100| 7.61e-07  2.04e-07  4.55e-08  7.07

7.066679913313681

In [36]:
np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)), \
np.log(np.linalg.det(A.value))

(0.013805802977064682,
 1.3756730383641288e-10,
 1.5780881929255095e-10,
 2.0000000137976546)

In [ ]:
solve_and_refine(problem, lsqr_iters=5000, iters=10, presolve=False)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
           Cone Program Solution Refinement          
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Q ∈ 𝗥^(8013 × 8013),  nnz(Q) = 7190,  ‖Q‖_F = 9.771e+01,

𝒦 = 𝗥^(5491) × 𝗥_+^(402) × 𝓢^(0) × 𝗦𝗗^(2060) × 𝒦_exp^*(20)

𝓢 is the product of 0 second-order cones
𝗦𝗗 is the product of 4 semi-definite cones
each 𝒦_exp ∈ 𝗥^3 is an exponential cone

           (Enzo Busseti 𝘦𝘵 𝘢𝘭., 2017-2019)            
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ite.   ‖𝒩(z^+)‖   ‖𝒩(z) + 𝗗𝛿‖   z_(n-1)  LSQR   time
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0	1.93e-04	nan	1e+00	0	0.00
1	1.14e-06	1.62e-07	1e+00	5000	4.46
2	6.64e-08	6.64e-08	1e+00	5000	9.23


In [33]:
np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)), \
np.log(np.linalg.det(A.value))

(0.013805341820329551,
 3.410493491161449e-11,
 1.1335974906350967e-10,
 2.000000004140919)

In [54]:
#mosek_opt, new_opt

In [85]:
new_opt = solve_and_refine(problem, iters = 5, lsqr_iters=1000, warm_start=False)
print((mosek_opt - new_opt)/ mosek_opt)
print(np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)),\
     np.log(np.linalg.det(A.value)) - 2)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 975, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 311, constraints m = 765
Cones:	primal zero / dual free vars: 400
	linear vars: 100
	sd vars: 265, sd blks: 2
Setup time: 1.13e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.90e+20  1.49e+20  1.00e+00 -5.47e+20  6.25e+20  1.80e+20  1.97e-03 
   100| 2.25e-03  6.53e-04  3.25e-06  5.25e+00  5.25e+00  9.21e-17  4.60e-02 
   200| 1.07e-03  4.08e-04  1.55e-

/Users/enzo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [90]:
B.value += np.random.randn(n,n) * 1E-4

new_opt = solve_and_refine(problem, iters = 2, lsqr_iters=1000, warm_start=True)
print((mosek_opt - new_opt)/ mosek_opt)
print(np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)),\
     np.log(np.linalg.det(A.value)) - 2)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Conic Solution Refinement              

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ite.    || N(z) ||_2      z[-1]     LSQR       time
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0	9.00e-04	1e+00	0	0.00
1	7.43e-08	1e+00	1000	0.15
2	9.16e-10	1e+00	1000	0.27
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Max num. refinement iters reached.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
-8.121645933996544e-07
-5.361927314068023e-12 9.764913864249008e-12 2.817849491158293e-11 -49.209431655219305


In [64]:
# new_opt = solve_and_refine(problem, iters = 2, lsqr_iters = 500, warm_start=True)
# print((mosek_opt - new_opt)/ mosek_opt)
# print(np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)))

In [67]:
# new_opt = solve_and_refine(problem, warm_start=True)
# print((mosek_opt - new_opt)/ mosek_opt)
# print(np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)))

In [58]:
# from cvxpy.reductions.solvers.solving_chain import construct_solving_chain

# solving_chain = construct_solving_chain(problem, solver = 'SCS')
# data, inverse_data = solving_chain.apply(problem)

# scs_solution = solving_chain.solve_via_data(problem, 
#                                             data=data, 
#                                             warm_start=True, 
#                                             verbose=True,
#                                             solver_opts = {'eps':1E-8})

# ## HERE GOES REFINE

# A, b, c, z_scs, dims = cvxpy_scs_to_csr(scs_solution, data)
# refined_z = cpr.refine(A,b,c, dims, z_scs, verbose=True)
# new_residual, u, v = cpr.residual_and_uv(refined_z, A, b, c, cpr.make_prod_cone_cache(dims))
# scs_solution['x'], scs_solution['s'], scs_solution['y'], \
#     tau, kappa = cpr.uv2xsytaukappa(u, v, A.shape[1])
# print(tau, kappa)

# cvxpy_solution = solving_chain.invert(scs_solution, inverse_data)

# problem.unpack_results(scs_solution, solving_chain, inverse_data)
# problem.value

In [14]:
data['dims']

In [23]:
refined_z = refine(A,b,c, dims, z_scs, verbose=True)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
               Conic Solution Refinement              

it.    ||N(z)||_2       z[-1]   LSQR  btrks     time
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0	8.89e-08	1e+00	  0	0	0.00
1	1.04e-08	1e+00	  30	0	0.05
2	6.82e-09	1e+00	  30	0	0.10
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Max num. refinement iters reached.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [33]:
refined_z = refine(A,b,c, dims, refined_z, verbose=True)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
               Conic Solution Refinement              

it.    ||N(z)||_2       z[-1]   LSQR  btrks     time
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0	2.90e-09	1e+00	  0	0	0.00
1	2.88e-09	1e+00	  30	0	0.04
2	2.87e-09	1e+00	  30	0	0.09
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Max num. refinement iters reached.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [184]:
problem.unpack_results?

In [122]:
my_solution

In [110]:
#solution

In [112]:
my_solution

In [97]:
data = probl.get_problem_data(solver='SCS')
solving_chain = data[1]

solving_chain.solve(probl, warm_start=True, solver_opts={}, verbose=True)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 8625, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 2731, constraints m = 6795
Cones:	primal zero / dual free vars: 3600
	linear vars: 900
	sd vars: 2295, sd blks: 2
Setup time: 2.35e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.07e+20  4.38e+20  1.00e+00 -2.81e+21  3.15e+21  9.37e+20  3.86e-03 
   100| 1.54e-02  2.35e-03  1.28e-04  8.46e+00  8.46e+00  3.01e-16  1.35e-01 
   200| 4.09e-03  6.27e-04  2

In [90]:
A, b, c, dims = data[0]['A'], data[0]['b'], data[0]['c'], data[0]['dims']
dims_dict = {}
dims_dict['l'] = int(dims.nonpos)
dims_dict['z'] = int(dims.zero)
dims_dict['q'] = list([int(el) for el in dims.soc])
dims_dict['s'] = list([int(el) for el in dims.psd])

TypeError: get_problem_data() got an unexpected keyword argument 'warm_start'

In [88]:
x_con, s_con, y_con, info = solve(A,b,c,dims_dict,
      max_lsqr_iters=1000, max_iters=2,
      #refine_solver_time_ratio=1.,
      verbose=True,
     solver_options={'verbose':True, 'eps':1e-6, 'max_iters':1000})


----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 8625, CG tol ~ 1/iter^(2.00)
eps = 1.00e-06, alpha = 1.50, max_iters = 1000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 2731, constraints m = 6795
Cones:	primal zero / dual free vars: 3600
	linear vars: 900
	sd vars: 2295, sd blks: 2
Setup time: 2.74e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.07e+20  4.38e+20  1.00e+00 -2.81e+21  3.15e+21  9.37e+20  4.60e-03 
   100| 1.54e-02  2.35e-03  1.28e-04  8.46e+00  8.46e+00  3.01e-16  1.88e-01 
   200| 4.09e-03  6.27e-04  2

In [89]:
data

({'A': <6795x2731 sparse matrix of type '<class 'numpy.float64'>'
  	with 8625 stored elements in Compressed Sparse Column format>,
  'b': array([-0., -0., -0., ...,  0.,  0.,  0.]),
  'c': array([1., 0., 0., ..., 0., 0., 0.]),
  'dims': <cvxpy.reductions.solvers.conic_solvers.conic_solver.ConeDims at 0xb2ab0feb8>,
  'offset': array([0.])},
  ({546: Variable((30, 30)), 1161: Variable((1830, 1)), 1162: Variable(())},
   {546: Variable((30, 30)),
    1161: Variable((60, 60), PSD=True),
    1162: Variable(())},
   {547: 547, 549: 549, 551: 551, 1163: 1163, 1165: 1165, 1167: 1167}),
  {'dims': <cvxpy.reductions.solvers.conic_solvers.conic_solver.ConeDims at 0xb2ab0feb8>,
   'eq_constr': [Zero(Expression(AFFINE, UNKNOWN, (30, 30))),
    Zero(Expression(AFFINE, UNKNOWN, (30, 30))),
    Zero(Expression(AFFINE, UNKNOWN, (30, 30))),
    Zero(Expression(AFFINE, UNKNOWN, (30, 30)))],
   'offset': 0.0,
   'other_constr': [NonPos(Expression(AFFINE, UNKNOWN, (30, 30))),
    PSD(Expression(AFFINE, UN

In [ ]:
data[1].apply

In [ ]:
data[1].accepts

In [ ]:
data[1].accepts

In [ ]:
data[1].solve

In [55]:
dir(data[1])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'accepts',
 'apply',
 'invert',
 'reductions',
 'solve',
 'solve_via_data',
 'solver']

In [31]:
#(B-A).value